# Set Up

Mount google drive, ignore if not using Google Collab:

In [1]:
from google.colab import drive

# mount google drive
drive.mount('/content/drive')
%cd /content/drive/My Drive
!git clone https://github.com/FranciscoLozCoding/cooling_with_code.git
%cd cooling_with_code
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
fatal: destination path 'cooling_with_code' already exists and is not an empty directory.
/content/drive/My Drive/cooling_with_code
Already up to date.


Import libraries

In [2]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# google collab
from google.colab import drive

#data science
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import pickle

#custom tools
from tools.environment import VALID_SPLIT, RANDOM_STATE

# Simple Model Development

This notebook is for creating a simple model for us to use as a baseline comparison when creating our future models. For details on the train/valid/test dataset refer to our past notebooks:
- [01_dataset_generation](/01_dataset_generation.ipynb)
- [02_more_dataset_generation](/02_more_dataset_generation.ipynb)

## Train and Validation Split


We will now split the training datasets into 70% training data and 30% validation data.

In [3]:
#load in traning dataset
fiftym_train = pd.read_csv('data/train/50m_buffer_dataset.csv')
hunm_train = pd.read_csv('data/train/100m_buffer_dataset.csv')
hunfifty_train = pd.read_csv('data/train/150m_buffer_dataset.csv')

# Split the data into features (X) and target (y), and then into training and validation sets
## 50m buffer zone dataset
fiftym_x = fiftym_train.drop(columns=['UHI']).values
fiftym_y = fiftym_train['UHI'].values
fiftym_x_train, fiftym_x_valid, fiftym_y_train, fiftym_y_valid = train_test_split(
    fiftym_x, fiftym_y,
    test_size=VALID_SPLIT,
    random_state=RANDOM_STATE)

## 100m buffer zone dataset
hunm_x = hunm_train.drop(columns=['UHI']).values
hunm_y = hunm_train['UHI'].values
hunm_x_train, hunm_x_valid, hunm_y_train, hunm_y_valid = train_test_split(
    hunm_x, hunm_y,
    test_size=VALID_SPLIT,
    random_state=RANDOM_STATE)

## 150m buffer zone dataset
hunfifty_x = hunfifty_train.drop(columns=['UHI']).values
hunfifty_y = hunfifty_train['UHI'].values
hunfifty_x_train, hunfifty_x_valid, hunfifty_y_train, hunfifty_y_valid = train_test_split(
    hunfifty_x, hunfifty_y,
    test_size=VALID_SPLIT,
    random_state=RANDOM_STATE)

## Feature Scaling

Next, we will normalize all columns so that none will overpower another because it has greater values. This is called Feature Scaling. We will use standard scaler since the linear model we will use works best with values that are standarized.

In [4]:
# Scale the training and validation data using standardscaler

## 50m buffer zone dataset
fiftym_sc = StandardScaler()
fiftym_x_train = fiftym_sc.fit_transform(fiftym_x_train)
fiftym_x_valid = fiftym_sc.transform(fiftym_x_valid)

## 100m buffer zone dataset
hunm_sc = StandardScaler()
hunm_x_train = hunm_sc.fit_transform(hunm_x_train)
hunm_x_valid = hunm_sc.transform(hunm_x_valid)

## 150m buffer zone dataset
hunfifty_sc = StandardScaler()
hunfifty_x_train = hunfifty_sc.fit_transform(hunfifty_x_train)
hunfifty_x_valid = hunfifty_sc.transform(hunfifty_x_valid)

## Model Training

Now that we have the data in a format suitable for machine learning, we can begin training a model. We will use a random forest regression model from the scikit-learn library. We chose this model because it is simple and it can provide us feedback on what features were important. We won't go too in depth in hyper parameters here since our aim is just to create a simple baseline model for comparison.

In [5]:
# Train the Random Forest model on the training data

## 50m buffer zone model
fiftym_model = RandomForestRegressor(
    n_estimators=100,
    random_state=RANDOM_STATE,
    criterion="squared_error")
fiftym_model.fit(fiftym_x_train, fiftym_y_train)

RandomForestRegressor(random_state=123)

In [6]:
## 100m buffer zone model
hunm_model = RandomForestRegressor(
    n_estimators=100,
    random_state=RANDOM_STATE,
    criterion="squared_error")
hunm_model.fit(hunm_x_train, hunm_y_train)

RandomForestRegressor(random_state=123)

In [7]:
## 150m buffer zone model
hunfifty_model = RandomForestRegressor(
    n_estimators=100,
    random_state=RANDOM_STATE,
    criterion="squared_error")
hunfifty_model.fit(hunfifty_x_train, hunfifty_y_train)

RandomForestRegressor(random_state=123)

## Model Evaluation

Now that we have trained our model, the next step is to evaluate its performance. For evaluation, we will use the R² score. This was chosen by the challenge authors as the score to evaluate participants models.

### In-Sample Evaluation

First, we will be evaluating our model's performance using the R² score on the training data. It is important to note that this is in-sample performance testing, which involves evaluating the model on the training dataset. These metrics are not truly indicative of the model's ability to generalize.

In [8]:
# Make predictions on the training data

## 50m buffer zone dataset
fiftym_insample_predictions = fiftym_model.predict(fiftym_x_train)

## 100m buffer zone dataset
hunm_insample_predictions = hunm_model.predict(hunm_x_train)

## 150m buffer zone dataset
hunfifty_insample_predictions = hunfifty_model.predict(hunfifty_x_train)

In [9]:
# calculate R-squared score for in-sample predictions
print(f"In-Sample Evaluation:")

## 50m buffer zone dataset
fiftym_insample_r2 = r2_score(fiftym_y_train, fiftym_insample_predictions)
print(f"  50m buffer zone R-squared: {fiftym_insample_r2}")

## 100m buffer zone dataset
hunm_insample_r2 = r2_score(hunm_y_train, hunm_insample_predictions)
print(f"  100m buffer zone R-squared: {hunm_insample_r2}")

## 150m buffer zone dataset
hunfifty_insample_r2 = r2_score(hunfifty_y_train, hunfifty_insample_predictions)
print(f"  150m buffer zone R-squared: {hunfifty_insample_r2}")

In-Sample Evaluation:
  50m buffer zone R-squared: 0.9798971030870426
  100m buffer zone R-squared: 0.9869424603623242
  150m buffer zone R-squared: 0.9900177365375015


### Out-Sample Evaluation

To estimate the out-of-sample performance, we will predict on the validation data now. Comparing the in-sample and out-sample evaluations can tell us if the model is overfitting.

In [10]:
# Make predictions on the validation data

## 50m buffer zone dataset
fiftym_outsample_predictions = fiftym_model.predict(fiftym_x_valid)

## 100m buffer zone dataset
hunm_outsample_predictions = hunm_model.predict(hunm_x_valid)

## 150m buffer zone dataset
hunfifty_outsample_predictions = hunfifty_model.predict(hunfifty_x_valid)

In [11]:
# calculate R-squared score for out-sample predictions
print(f"Out-Sample Evaluation:")

## 50m buffer zone dataset
fiftym_outsample_r2 = r2_score(fiftym_y_valid, fiftym_outsample_predictions)
print(f"  50m buffer zone R-squared: {fiftym_outsample_r2}")

## 100m buffer zone dataset
hunm_outsample_r2 = r2_score(hunm_y_valid, hunm_outsample_predictions)
print(f"  100m buffer zone R-squared: {hunm_outsample_r2}")

## 150m buffer zone dataset
hunfifty_outsample_r2 = r2_score(hunfifty_y_valid, hunfifty_outsample_predictions)
print(f"  150m buffer zone R-squared: {hunfifty_outsample_r2}")

Out-Sample Evaluation:
  50m buffer zone R-squared: 0.8622465452118891
  100m buffer zone R-squared: 0.9116159865712685
  150m buffer zone R-squared: 0.9313469521366144


# Challenge Submission

We will now submit our simple `RandomForestRegressor` model to the challenge. Since our 150m buffer zone RandomForestRegressor model did the best we will use this model. Remember, this is just for comparison so this is not our final model.
>NOTE: We can have multiple submissions.

In [13]:
# Reading the test dataset for the submission
hunfifty_test = pd.read_csv('data/test/150m_buffer_test_dataset.csv')

# Remove lat and lon, for now will be combined with predicted UHI later for submission
lon = hunfifty_test['Longitude'].values
lat = hunfifty_test['Latitude'].values
hunfifty_test = hunfifty_test.drop(columns=['Longitude', 'Latitude']).values

# Feature Scaling
hunfifty_x_test = hunfifty_sc.transform(hunfifty_test)

# Model prediction
hunfifty_test_predictions = hunfifty_model.predict(hunfifty_x_test)

# combine predictions with lat and lon
submission_df = pd.DataFrame({
    'Longitude': lon,
    'Latitude': lat,
    'UHI Index': hunfifty_test_predictions
})
submission_df

Save the submission dataframe to submit into challenge portal:

In [15]:
submission_df.to_csv("150m_simple_RandomForestRegressor_model_submission.csv", index=False)

# Summary Table

Here we will provide a summary table of the simple models, for quick comparison for our future models.

In [16]:
# create the summary table
models = [
    '50m_simple_RandomForestRegressor_model',
    '100m_simple_RandomForestRegressor_model',
    '150m_simple_RandomForestRegressor_model']

data = {
    'Model': models,
    'Training R-squared': [fiftym_insample_r2, hunm_insample_r2, hunfifty_insample_r2],
    'Validation R-squared': [fiftym_outsample_r2, hunm_outsample_r2, hunfifty_outsample_r2]
}

summary_df = pd.DataFrame(data)
summary_df

,Model,Training R-squared,Validation R-squared
0,50m_simple_RandomForestRegressor_model,0.979897,0.862247
1,100m_simple_RandomForestRegressor_model,0.986942,0.911616
2,150m_simple_RandomForestRegressor_model,0.990018,0.931347


# Save The Model

Finally, we will save all three models so we don't have to retrain them. This will also allow us to use them in other notebooks.

In [17]:
# Save the models to files

## 50m buffer zone model
with open('50m_simple_RandomForestRegressor_model.pkl', 'wb') as f:
    pickle.dump(fiftym_model, f)
with open('50m_standard_scaler.pkl', 'wb') as f:
    pickle.dump(fiftym_sc, f)

## 100m buffer zone model
with open('100m_simple_RandomForestRegressor_model.pkl', 'wb') as f:
    pickle.dump(hunm_model, f)
with open('100m_standard_scaler.pkl', 'wb') as f:
    pickle.dump(hunm_sc, f)

## 150m buffer zone model
with open('150m_simple_RandomForestRegressor_model.pkl', 'wb') as f:
    pickle.dump(hunfifty_model, f)
with open('150m_standard_scaler.pkl', 'wb') as f:
    pickle.dump(hunfifty_sc, f)

# Conclusion

Now that we have our simple models developed we can move on to Exploratory Data Analysis to provide us a starting point for developing more complex models...